In [2]:
!pip install requests beautifulsoup4 pandas transformers torch

   ---------------------------------------- 0.0/159.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/159.8 MB 991.0 kB/s eta 0:02:42
   ---------------------------------------- 0.0/159.8 MB 991.0 kB/s eta 0:02:42
   ---------------------------------------- 0.1/159.8 MB 1.1 MB/s eta 0:02:30
   ---------------------------------------- 0.2/159.8 MB 1.1 MB/s eta 0:02:29
   ---------------------------------------- 0.2/159.8 MB 1.2 MB/s eta 0:02:18
   ---------------------------------------- 0.3/159.8 MB 1.2 MB/s eta 0:02:15
   ---------------------------------------- 0.4/159.8 MB 1.3 MB/s eta 0:02:05
   ---------------------------------------- 0.5/159.8 MB 1.3 MB/s eta 0:01:59
   ---------------------------------------- 0.5/159.8 MB 1.2 MB/s eta 0:02:10
   ---------------------------------------- 0.6/159.8 MB 1.5 MB/s eta 0:01:49
   ---------------------------------------- 0.7/159.8 MB 1.5 MB/s eta 0:01:46
   ---------------------------------------- 0.8/159.8 MB 1.6 MB/s e

In [3]:
import requests
from bs4 import BeautifulSoup
import os
import pandas as pd
from datetime import datetime, timedelta
from transformers import pipeline

In [6]:
def extract_header_and_url(anchors, category="general"):
    header_urls = []
    for anchor in anchors:
        header_urls.append({"title": anchor["title"],
                            "url": anchor["href"],
                            "type": category})
    return header_urls

In [7]:
def category_news(categorical_news):
    categorical_news_data = []
    for news in categorical_news:
        news_category = list(news.keys())[0]
        if news_category != "Startups":
            news_url = news[news_category]
            response = requests.get(news_url)
            soup = BeautifulSoup(response.content, 'html.parser')
            category_list = soup.find(id="cagetory")

            lis = category_list.find_all("li", {"class": "clearfix"})
            anchors = [li.find("a") for li in lis]
            categorical_news_data = categorical_news_data + extract_header_and_url(anchors, news_category)
    return categorical_news_data

In [8]:
def general_news(base_url):
    response = requests.get(base_url)
    soup = BeautifulSoup(response.content, "html.parser")
    bizs = soup.find_all("div", {"class": "bx"})
    left_content = soup.find("section", {"class": "main-left"})

    main_headers = left_content.find("div", {"class": "mob-hide"})
    more_news = left_content.find("div", {"class": "clearfix MT20 mob-hide"})

    main_headers = main_headers.find("div", {"class": "clearfix"})
    left_panel = main_headers.find("div", {"class": "sub-col-left"})
    right_panel = main_headers.find("div", {"class": "sub-col-rht"})

    left_panel_news = left_panel.find("div", {"class": "clearfix ltsnewsbx"})
    left_panel_news_headers = left_panel_news.find_all("a")
    right_panel_news = right_panel.find("div", {"class": "clearfix listed_newsec"})
    right_panel_news_headers = right_panel_news.find_all("a")

    left_news_data = extract_header_and_url(left_panel_news_headers)
    right_news_data = extract_header_and_url(right_panel_news_headers)

    more_news_left_section = more_news.find(id="keynwstb1")

    more_news_data = more_news_left_section.find_all("a")
    more_news_data = extract_header_and_url(more_news_data)
    main_news_data = left_news_data + right_news_data + more_news_data

    categories = []
    for biz in bizs:
        category = biz.find("h2").find("a")
        category_name = category.get_text()
        category_url = category["href"]
        categories.append({category_name: category_url})
    return categories, main_news_data

In [9]:
def fetch_content(content_url):
    response = requests.get(content_url)
    if response:
        soup = BeautifulSoup(response.content, "html.parser")
        content_data = soup.find(id="contentdata")
        if content_data:
            paras = content_data.find_all("p")
            return paras
    return None

In [10]:
def analyze_sentiment(text):
    result = sentiment_analyzer(text)
    return result[0]['label']

In [13]:
def create_csv_file(content, name):
    if not os.path.exists(OUTPUT_DIR):
        os.makedirs(OUTPUT_DIR)
    date = datetime.now().strftime('%Y-%m-%d')
    df = pd.DataFrame(content)
    df.to_csv(os.path.join(OUTPUT_DIR, f'{name}_{date}.csv'), index=False)

In [14]:
sentiment_analyzer = pipeline('sentiment-analysis')
BASE_URL = 'https://www.moneycontrol.com'
DAYS = 2
OUTPUT_DIR = 'data/raw'

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


RuntimeError: At least one of TensorFlow 2.0 or PyTorch should be installed. To install TensorFlow 2.0, read the instructions at https://www.tensorflow.org/install/ To install PyTorch, read the instructions at https://pytorch.org/.

In [12]:
categories, anchors = general_news(BASE_URL)
categorical_news_data = category_news(categories)

final_news_urls = anchors + categorical_news_data
create_csv_file(final_news_urls, "content_urls")

content_data = []
for article_url in final_news_urls:
    if article_url["url"].startswith("https://www.moneycontrol.com"):
        content = fetch_content(article_url["url"])
        if content:
            content = ' '.join(p.get_text() for p in content)
            article_url["content"] = content
            content_data.append(article_url)

create_csv_file(content_data, "content_data")

NameError: name 'BASE_URL' is not defined

In [10]:
import os
abs_path = os.path.abspath('data/raw')
print(abs_path)  # Outputs the absolute path

C:\Users\nandk\Desktop\PYTHON PRACTICE\data\raw


In [ ]:
os.path